In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_path = "/home/clairegayral/Documents/openclassroom/data/"

## my .py : 
import read_csv
import preprocess
import list_from_data_field 
import missing_values_treatment

My preprocess consisted in : 
* Read the data : I only kept data with the "nutrition score" for this analyse,
* Drop the columns with too many missing values
* Select the variables that will interest me : the nutrition variable, with name of the product, its code reference, country of sell, and name of the person that inter it in the open food fact base.
* Merge the two columns that are about the palm oil ingredients.
* Remove outliers (from the semantic of columns, and compared to the other values)

Mon prétraitement consistait en : 
* Lecture des données : Je n'ai gardé que les données avec le "score nutritionnel" pour cette analyse,
* Suppression des colonnes avec trop de valeurs manquantes
* Sélection des variables d'intérêt : la variable nutritionnelle, avec le nom du produit, sa référence de code, le pays de vente, et le nom de la personne qui l'a introduit dans la base de données 'open food facts'.
* Fusion des deux colonnes qui concernent les ingrédients de l'huile de palme.
* Fusion des doublons (par nom de produit)
* Suppression les valeurs aberrantes (de la sémantique des colonnes, et par rapport aux autres valeurs).


In [78]:
##
## read data : 
##
# read_csv.save_csv()
df_original = pd.read_csv(data_path+"projet2/df_original.csv", 
                          low_memory=False, index_col=0)

df = df_original.copy()

##
## select variables 
##
list_of_nutri_facts = list_from_data_field.list_of_nutri_facts
interest_var = pd.Index(["code","product_name","creator","countries",
                         "additives_n","ingredients_from_palm_oil"])
interest_var = interest_var.append(df.columns.intersection(list_of_nutri_facts))
df = preprocess.select_columns(df, interest_var)

##
## set dtype : 
##
df = preprocess.set_dtypes(df)

##
## merge "from palm oil" 
##
df = preprocess.merge_palm_oil_cols(df)

##
## drop empty columns
##
nan_repartition = df_original.isna().sum(axis=0)
# nan_threshold = nan_repartition.mean()
nan_threshold = nan_repartition.quantile(0.75)
df = preprocess.preprocess_drop_col_nan(df_original, nan_threshold)


In [79]:
float_var = list_from_data_field.list_of_nutri_facts
float_var += ["additives_n", "ingredients_from_palm_oil","ingredients_from_palm_oil_n"]
float_var = df.columns.intersection(float_var).values

data = df.copy()
## std var by var :
data_float = data[float_var].copy()
data_float = preprocessing.StandardScaler().fit_transform(data_float)



products = data["product_name"].drop_duplicates()
for product in products[:6] : 
    x = data[data.product_name == product]
    product_index =  x.index
    if len(product_index) > 1 : 
        p_float_var = (x.notna().sum()>=1).index.intersection(float_var)
        p_var_index = x[p_float_var].columns
        product_values = x.loc[:,p_float_var]
        product_score = abs(product_values.diff().iloc[1:]/product_values.mean()).mean().mean()
#         if product_score < 0.7 :
            #### pb avec le .at, marche sans mais erreur
#             res = pd.Series(x.drop_duplicates(subset="product_name").values[0], index=x.columns)
#             res[p_var_index] = product_values.fillna(method="bfill",
#                                     axis=0).fillna(method="ffill",axis=0).mean().values
#             print(product,res)
#     tmp = data[data.product_name == product]
x.drop_duplicates(subset="product_name")

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,phylloquinone_100g
63,0000000005005,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1521663684,2018-03-21T20:21:24Z,1545934272,2018-12-27T18:11:12Z,Abondance,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN


In [12]:
product_values

,additives_n,ingredients_from_palm_oil,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,...,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
4,0.0,NaN,936.0,NaN,936.0,8.2,2.2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,NaN


In [50]:
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn import preprocessing

products = data["product_name"].drop_duplicates()
product = products.iloc[1]

x = data[data.product_name == product]
product_index =  x.index

if len(product_index) > 1 : 
    p_float_var = (x.notna().sum()>=1).index.intersection(float_var)
    p_var_index = x[p_float_var].columns
    product_values = x.loc[:,p_float_var]

    p_dist = nan_euclidean_distances(product_values)

In [49]:
nan_euclidean_distances(product_values)

array([[  0.        , 527.48694793, 461.54388638],
       [527.48694793,   0.        ,  74.56698992],
       [461.54388638,  74.56698992,   0.        ]])

In [31]:
x.columns.shape

(65,)

In [41]:
x.drop_duplicates(subset="product_name").values[0]

array(['0000000000949', 'kiliweb', 'Salade de carottes râpées', 'France',
       nan, nan, nan, 32.0, 134.0, 0.3, 0.1, nan, nan, nan, nan, nan, nan,
       nan, 5.3, 3.9, nan, nan, nan, 0.9, nan, nan, nan, 0.168, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, 1.0, nan], dtype=object)

In [ ]:

#     res = pd.Series(x.drop_duplicates(subset="product_name").values[0], index=x.columns)
#     res[p_var_index] = product_values.fillna(method="bfill",
#                             axis=0).fillna(method="ffill",axis=0).mean().values
    
    

In [ ]:
p_float_var = (x.notna().sum()>=1).index.intersection(float_var)

res = x.drop_duplicates(subset="product_name")
res[p_float_var] = product_values.fillna(method="bfill",axis=0).fillna(method="ffill",axis=0).mean()
# print(res[p_float_var])

In [82]:
data.loc[:,float_var]

,additives_n,ingredients_from_palm_oil_n,energy-kj_100g,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,...,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,carbon-footprint_100g,nutrition-score-fr_100g
4,-0.711746,-0.139854,-0.053007,NaN,-0.015797,-0.006382,-0.001500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.990612
15,NaN,NaN,NaN,-0.061308,-0.062512,-0.012700,-0.001668,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.919729
24,NaN,NaN,NaN,0.021267,0.022530,0.004653,-0.000432,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.541120
34,NaN,NaN,NaN,-0.031733,-0.032048,-0.012940,-0.001676,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.256848
38,NaN,NaN,NaN,-0.035045,-0.035485,-0.002784,-0.001596,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.919729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716261,NaN,NaN,NaN,0.012986,0.014026,0.008651,-0.000311,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.878239
1716263,NaN,NaN,NaN,0.014169,0.015249,-0.012540,-0.001676,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.032102
1716267,NaN,NaN,NaN,0.009910,0.010822,0.003214,-0.001195,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.653493
1716269,-0.052882,-0.139854,-0.056121,NaN,-0.016438,-0.006862,-0.001291,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.316374


In [ ]:
data.iloc[(data[float_var].isna().sum(axis=0)==0)]

In [ ]:
tmp = df.groupby('product_name').fillna(method='ffill').\
                             fillna(method='bfill').\
                             drop_duplicates()
                     


In [74]:
data[float_var].isna().sum(axis=0)==0

additives_n                    False
ingredients_from_palm_oil      False
energy-kj_100g                 False
energy-kcal_100g               False
energy_100g                    False
                               ...  
ph_100g                        False
fruits-vegetables-nuts_100g    False
carbon-footprint_100g          False
nutrition-score-fr_100g         True
nutrition-score-uk_100g        False
Length: 61, dtype: bool

In [ ]:
float_var = list_from_data_field.list_of_nutri_facts
float_var += ["additives_n", "ingredients_from_palm_oil","ingredients_from_palm_oil_n"]
float_var = df.columns.intersection(float_var).values

product = products.iloc[1]
product_index =  data[data.product_name == product].index
if len(product_index) > 1 : 
    product_values = data.loc[product_index,(data.loc[product_index].notna().sum()>=1)]
#     meth 1 
    product_mean = product_values[product_values.columns.intersection(float_var)].mean()
    product_std = product_values[product_values.columns.intersection(float_var)].std()
    to_merge = ((product_mean - product_std) - product_values < 0) | (product_values - (product_mean - product_std) < 0)
    # to_merge.sum().sum()/to_merge.shape[1]
    product_diff = product_values[product_values.columns.intersection(float_var)].diff()#.iloc[1:]
    print(abs(product_diff/product_values.mean()).mean().mean())

In [ ]:
float_var = list_from_data_field.list_of_nutri_facts
float_var += ["additives_n", "ingredients_from_palm_oil","ingredients_from_palm_oil_n"]
float_var = df.columns.intersection(float_var).values

data = df.copy()
products = data["product_name"].drop_duplicates()
for product in products : 
    product_index =  data[data.product_name == product].index
    if len(product_index) > 1 : 
        product_values = data.loc[product_index,(data.loc[product_index].notna().sum()>=1)]
    #     meth 1 
        product_mean = product_values[product_values.columns.intersection(float_var)].mean()
        product_std = product_values[product_values.columns.intersection(float_var)].std()
        to_merge = ((product_mean - product_std) - product_values < 0) | (product_values - (product_mean - product_std) < 0)
        # to_merge.sum().sum()/to_merge.shape[1]


In [ ]:
product_diff

In [ ]:
float_var = list_from_data_field.list_of_nutri_facts
float_var += ["additives_n", "ingredients_from_palm_oil","ingredients_from_palm_oil_n"]
float_var = df.columns.intersection(float_var).values

import missing_values_treatment
##
## from the hyp that the variable has been entered in mg instead of g -> rescale 
##
df = missing_values_treatment.rescale_outliers100g_val(df)

##
## drop outliers values and drop product/variables with too many NaNs
##
df = missing_values_treatment.drop_outliers(df)
df.shape

In [ ]:
nb_line_plot = int(np.floor(len(float_var)/2)+1)
fig = plt.figure(figsize=(16, 100))

fig_count = 1
for var in df.columns.intersection(float_var) :#data.columns.intersection(list_of_nutri_facts):
    ax = fig.add_subplot(nb_line_plot,2, fig_count)
    nb_bins = min(20, len(np.unique(df[var].dropna().values)))
    ax.hist(df[var], bins = nb_bins, color='steelblue', density=True, edgecolor='none')
    ax.set_title(var)
    fig_count += 1
plt.show()

In [ ]:
data = df.copy()
colname = "additives_n"
possible_vals = possible_val_dict[colname]


help_to_set_outliers_vals(data, colname, possible_vals)

In [ ]:
import list_from_data_field
float_var = list_from_data_field.list_of_characteristics 

floa

In [ ]:
def merge_palm_oil_cols(data):
    if "ingredients_from_palm_oil_n" in data.columns :
        data.at[data["ingredients_from_palm_oil_n"] > 0, "ingredients_from_palm_oil_n"] = 1
        if ("ingredients_from_palm_oil" in data.columns) :
            data['ingredients_from_palm_oil'].fillna(data['ingredients_from_palm_oil_n'], inplace=True)
            data.drop("ingredients_from_palm_oil_n", inplace = True, axis = 1)
        else :
             data.rename(columns={"ingredients_from_palm_oil_n":"ingredients_from_palm_oil"}, inplace=True)
        data["ingredients_from_palm_oil"] = data["ingredients_from_palm_oil"].astype("float")
    return(data)